In [ ]:
import sys
sys.path.append('..')

# Helper imports
from data import CIFAR10, IMAGENETTE
import utils

# Numerical computing and display imports
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

# Other imports
from datetime import datetime
import logging
import pickle
import os

In [ ]:
# Load and Prepare the Data
folder_path = '../../images/'
dataset = 'imagenette'    
if dataset == 'cifar10':
    images = CIFAR10()
if dataset == 'imagenette':
    images = IMAGENETTE()
x = images.load()
images.save_to_disk(x, folder_path, num_images=110)
folder_path = utils.move_data_to_temp_ram(folder_path, ram_size_mb=50)

In [ ]:
hamm_distance = [[], []]
hash_funcs = ['neuralhash', 'pdq']
hash_lengths = [128, 256]

for i, hash_func in enumerate(hash_funcs):
    print(f'PHA {i+1}: {hash_func}')
    for j in range(5):
        target_path = f'../../images/{105+(j+1)}.bmp' 
        target_hash = utils.compute_hash(target_path, hash_func=hash_func)

        for k in range(100):
            img_path = f'../../images/{k+1}.bmp' 
            img_hash = utils.compute_hash(img_path, hash_func=hash_func)
            # Compute the hamming distance between the images
            hamm = utils.distance(img_hash, target_hash, type='hamming', hash_func=hash_func)
            hamm_distance[i].append(hamm/hash_lengths[i])
            print(f'{j*50 + (k+1)}: Relative Hamming Distance {(hamm/hash_lengths[i])*100:.2f}%')
    print(f'Average Relative Hamming Distance: {sum(hamm_distance[i])/len(hamm_distance[i]) :.2f}')

In [ ]:
neuralhash_hamm_distance = sorted(hamm_distance[0])
count, bins_count = np.histogram(neuralhash_hamm_distance, bins=500) 
pdf = count / sum(count)
cdf = np.cumsum(pdf)
plt.plot(bins_count[1:], cdf, label="NeuralHash") 

pdq_hamm_distance = sorted(hamm_distance[1])
count, bins_count = np.histogram(pdq_hamm_distance, bins=500) 
pdf = count / sum(count)
cdf = np.cumsum(pdf)
plt.plot(bins_count[1:], cdf, label="PDQ") 
plt.xlabel('Hamming Distances')
plt.ylabel('Probabilities')
plt.legend()